In [55]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
from pandas import DataFrame
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import GroupKFold
import lightgbm as lgb 
from sklearn.metrics import root_mean_squared_error
import random
random.seed(42)
import pickle
pd.options.mode.chained_assignment = None

In [ ]:
# os.chdir('')

#### Import Data

In [56]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sub = pd.read_csv('SampleSubmission.csv')

#### Data Observation

In [57]:
# scan briefly through the train
train.head()

,id,site_id,site_latitude,site_longitude,city,country,date,hour,sulphurdioxide_so2_column_number_density,sulphurdioxide_so2_column_number_density_amf,...,cloud_cloud_top_height,cloud_cloud_base_pressure,cloud_cloud_base_height,cloud_cloud_optical_depth,cloud_surface_albedo,cloud_sensor_azimuth_angle,cloud_sensor_zenith_angle,cloud_solar_azimuth_angle,cloud_solar_zenith_angle,pm2_5
0,id_vjcx08sz91,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-10-25,13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.015000
1,id_bkg215syli,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-02,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.267200
2,id_oui2pot3qd,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-03,13,NaN,NaN,...,6791.682888,51171.802486,5791.682829,11.816715,0.192757,-96.411890,61.045123,-121.307414,41.898269,39.450741
3,id_9aandqzy4n,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-08,14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.537600
4,id_ali5x2m4iw,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-09,13,0.000267,0.774656,...,1451.050659,96215.906250,451.050598,10.521009,0.153114,-97.811241,49.513439,-126.064453,40.167355,19.431731


In [58]:
# scan briefly through the test

In [59]:
# check the shape of train
print(f'Number of rows and columns for train: {train.shape}')


# check the shape of test


Number of rows and columns for train: (8071, 80)


In [60]:
# check for NaN values, empty rows and columns, count of NaN values for each column

# for train
train.isna().sum()

id                               0
site_id                          0
site_latitude                    0
site_longitude                   0
city                             0
                              ... 
cloud_sensor_azimuth_angle    3077
cloud_sensor_zenith_angle     3077
cloud_solar_azimuth_angle     3077
cloud_solar_zenith_angle      3077
pm2_5                            0
Length: 80, dtype: int64

In [61]:
# check for columns with nan values greater than a specific value

count_of_nans = 0

column_indices = np.array(np.where(train.isna().sum() > count_of_nans)).flatten()

# select the columns from train using their index
train_nan = train.iloc[:, column_indices]

# get the total number of nan values for each column
train_nan.isna().sum()


sulphurdioxide_so2_column_number_density          4912
sulphurdioxide_so2_column_number_density_amf      4912
sulphurdioxide_so2_slant_column_number_density    4912
sulphurdioxide_cloud_fraction                     4912
sulphurdioxide_sensor_azimuth_angle               4912
                                                  ... 
cloud_surface_albedo                              3077
cloud_sensor_azimuth_angle                        3077
cloud_sensor_zenith_angle                         3077
cloud_solar_azimuth_angle                         3077
cloud_solar_zenith_angle                          3077
Length: 70, dtype: int64

In [62]:
# if you want to see all the names of the columns 
print(train_nan.columns.tolist())

['sulphurdioxide_so2_column_number_density', 'sulphurdioxide_so2_column_number_density_amf', 'sulphurdioxide_so2_slant_column_number_density', 'sulphurdioxide_cloud_fraction', 'sulphurdioxide_sensor_azimuth_angle', 'sulphurdioxide_sensor_zenith_angle', 'sulphurdioxide_solar_azimuth_angle', 'sulphurdioxide_solar_zenith_angle', 'sulphurdioxide_so2_column_number_density_15km', 'carbonmonoxide_co_column_number_density', 'carbonmonoxide_h2o_column_number_density', 'carbonmonoxide_cloud_height', 'carbonmonoxide_sensor_altitude', 'carbonmonoxide_sensor_azimuth_angle', 'carbonmonoxide_sensor_zenith_angle', 'carbonmonoxide_solar_azimuth_angle', 'carbonmonoxide_solar_zenith_angle', 'nitrogendioxide_no2_column_number_density', 'nitrogendioxide_tropospheric_no2_column_number_density', 'nitrogendioxide_stratospheric_no2_column_number_density', 'nitrogendioxide_no2_slant_column_number_density', 'nitrogendioxide_tropopause_pressure', 'nitrogendioxide_absorbing_aerosol_index', 'nitrogendioxide_cloud_f

In [63]:
# do same for the test set


##### How does the train compare with the test data ?

In [64]:
# Are the site_ids in the train same as those in the test



In [65]:
# Do we have the same city in train as those in the test



### Data Cleaning

In [66]:
# concatenate both train and test to make it easier cleaning

# assign the pm2_5 column to test
test['pm2_5'] = float(-1)

# concatenate
full_data = pd.concat([train, test]).reset_index(drop=True)

full_data

,id,site_id,site_latitude,site_longitude,city,country,date,hour,sulphurdioxide_so2_column_number_density,sulphurdioxide_so2_column_number_density_amf,...,cloud_cloud_top_height,cloud_cloud_base_pressure,cloud_cloud_base_height,cloud_cloud_optical_depth,cloud_surface_albedo,cloud_sensor_azimuth_angle,cloud_sensor_zenith_angle,cloud_solar_azimuth_angle,cloud_solar_zenith_angle,pm2_5
0,id_vjcx08sz91,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-10-25,13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.015000
1,id_bkg215syli,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-02,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.267200
2,id_oui2pot3qd,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-03,13,NaN,NaN,...,6791.682888,51171.802486,5791.682829,11.816715,0.192757,-96.411890,61.045123,-121.307414,41.898269,39.450741
3,id_9aandqzy4n,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-08,14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.537600
4,id_ali5x2m4iw,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-09,13,0.000267,0.774656,...,1451.050659,96215.906250,451.050598,10.521009,0.153114,-97.811241,49.513439,-126.064453,40.167355,19.431731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10849,id_5yblexf7zp,64654b4dd5320e001d6c499a,-0.10098,34.76242,Kisumu,Kenya,2024-01-27,10,0.000099,0.660385,...,4975.111702,67328.459370,3975.111746,2.831732,0.250637,72.428178,55.404121,-150.645025,21.248325,-1.000000
10850,id_c9cycuvetl,65c8c557b3d86f0012b2e32b,3.89696,11.50603,Yaoundé,Cameroon,2024-02-21,12,NaN,NaN,...,3362.649565,77172.040801,2362.649565,13.719315,0.228072,-101.384688,16.577282,-125.511751,25.086502,-1.000000
10851,id_8xu1ardxni,65c8c557b3d86f0012b2e32b,3.89696,11.50603,Yaoundé,Cameroon,2024-02-22,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000
10852,id_aumy97t1iu,65c8c557b3d86f0012b2e32b,3.89696,11.50603,Yaoundé,Cameroon,2024-02-23,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000


In [70]:
# Drop columns that have more than 70% of their rows being NaN values

percentage = 0.50 # set percentage

dropCol = (full_data.isnull().sum()/len(full_data))[(full_data.isnull().sum()/len(full_data)).gt(percentage)].index.to_list()

print(f'columns that have more than {percentage * 100}% of their rows being NaN values: {dropCol}')

columns that have more than 50.0% of their rows being NaN values: ['sulphurdioxide_so2_column_number_density', 'sulphurdioxide_so2_column_number_density_amf', 'sulphurdioxide_so2_slant_column_number_density', 'sulphurdioxide_cloud_fraction', 'sulphurdioxide_sensor_azimuth_angle', 'sulphurdioxide_sensor_zenith_angle', 'sulphurdioxide_solar_azimuth_angle', 'sulphurdioxide_solar_zenith_angle', 'sulphurdioxide_so2_column_number_density_15km', 'nitrogendioxide_no2_column_number_density', 'nitrogendioxide_tropospheric_no2_column_number_density', 'nitrogendioxide_stratospheric_no2_column_number_density', 'nitrogendioxide_no2_slant_column_number_density', 'nitrogendioxide_tropopause_pressure', 'nitrogendioxide_absorbing_aerosol_index', 'nitrogendioxide_cloud_fraction', 'nitrogendioxide_sensor_altitude', 'nitrogendioxide_sensor_azimuth_angle', 'nitrogendioxide_sensor_zenith_angle', 'nitrogendioxide_solar_azimuth_angle', 'nitrogendioxide_solar_zenith_angle', 'uvaerosollayerheight_aerosol_height'

In [14]:
# drop these columns
full_data = full_data.drop(dropCol, axis=1).reset_index(drop=True)

full_data.head()

,id,site_id,site_latitude,site_longitude,city,country,date,hour,sulphurdioxide_so2_column_number_density,sulphurdioxide_so2_column_number_density_amf,...,cloud_cloud_top_height,cloud_cloud_base_pressure,cloud_cloud_base_height,cloud_cloud_optical_depth,cloud_surface_albedo,cloud_sensor_azimuth_angle,cloud_sensor_zenith_angle,cloud_solar_azimuth_angle,cloud_solar_zenith_angle,pm2_5
0,id_vjcx08sz91,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-10-25,13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.015000
1,id_bkg215syli,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-02,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.267200
2,id_oui2pot3qd,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-03,13,NaN,NaN,...,6791.682888,51171.802486,5791.682829,11.816715,0.192757,-96.411890,61.045123,-121.307414,41.898269,39.450741
3,id_9aandqzy4n,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-08,14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.537600
4,id_ali5x2m4iw,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-09,13,0.000267,0.774656,...,1451.050659,96215.906250,451.050598,10.521009,0.153114,-97.811241,49.513439,-126.064453,40.167355,19.431731


In [15]:
# check if the data still contain nan values

def count_nan_values(data:DataFrame, count_of_nans:int=0)-> DataFrame :

    column_indices = np.array(np.where(data.isna().sum() > count_of_nans)).flatten()

    # select the columns from train using their index
    data_nan = data.iloc[:, column_indices]

    # get the total number of nan values for each column
    columns = data_nan.isna().sum().index.tolist()
    nan_counts = data_nan.isna().sum().values.tolist()

    data = {'column_names': columns, 'nan_counts': nan_counts}

    return pd.DataFrame(data)


nan_df = count_nan_values(full_data, 0)
nan_df.head()

,column_names,nan_counts
0,sulphurdioxide_so2_column_number_density,5986
1,sulphurdioxide_so2_column_number_density_amf,5986
2,sulphurdioxide_so2_slant_column_number_density,5986
3,sulphurdioxide_cloud_fraction,5986
4,sulphurdioxide_sensor_azimuth_angle,5986


In [16]:
# fill the nan values using forward-fill and backward-fill

# get the nan columns remaining in the full_data
nan_cols = nan_df.column_names.tolist()
nan_cols

['sulphurdioxide_so2_column_number_density',
 'sulphurdioxide_so2_column_number_density_amf',
 'sulphurdioxide_so2_slant_column_number_density',
 'sulphurdioxide_cloud_fraction',
 'sulphurdioxide_sensor_azimuth_angle',
 'sulphurdioxide_sensor_zenith_angle',
 'sulphurdioxide_solar_azimuth_angle',
 'sulphurdioxide_solar_zenith_angle',
 'sulphurdioxide_so2_column_number_density_15km',
 'carbonmonoxide_co_column_number_density',
 'carbonmonoxide_h2o_column_number_density',
 'carbonmonoxide_cloud_height',
 'carbonmonoxide_sensor_altitude',
 'carbonmonoxide_sensor_azimuth_angle',
 'carbonmonoxide_sensor_zenith_angle',
 'carbonmonoxide_solar_azimuth_angle',
 'carbonmonoxide_solar_zenith_angle',
 'nitrogendioxide_no2_column_number_density',
 'nitrogendioxide_tropospheric_no2_column_number_density',
 'nitrogendioxide_stratospheric_no2_column_number_density',
 'nitrogendioxide_no2_slant_column_number_density',
 'nitrogendioxide_tropopause_pressure',
 'nitrogendioxide_absorbing_aerosol_index',
 '

In [17]:
for col in nan_cols:
        while full_data[col].isnull().sum()>0:
                full_data[col] = full_data[col].fillna(full_data[["site_id", col]].groupby(["site_id"]).shift(periods=0).ffill(limit=1).bfill(limit=1)[col])


In [18]:
# check if the data still has NaN values
count_nan_values(full_data, 0)


,column_names,nan_counts


#### Export Data

In [19]:
# Export data

full_data.to_csv('full_data.csv')